In [18]:
import deribit_data as dm
from option import Option
import numpy as np
from utils import *
from sklearn.metrics import mean_squared_error

In [19]:
data = dm.Scraper(currency='BTC')

In [20]:
hist_vol = data.get_hist_vol(save_csv=False)

In [21]:
raw_df = data.collect_data(max_workers = 10, save_csv = False)

Data Collected


In [22]:
option = Option(raw_df, 'C')

/Users/pierrebelamri/Documents/GitHub/deribit_scraping/utils.py:104: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/pierrebelamri/Documents/GitHub/deribit_scraping/utils.py:128: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/pierrebelamri/Documents/GitHub/deribit_scraping/utils.py:129: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

In [23]:
option.append_imp_vol_to_df()

In [24]:
option.df.head()

,S,K,V,_T,bids,asks,last_price,mid_iv,option_type,mid,moneyness,I_VOL
158,32897.84,32000.0,41.4,0.000822,0.035984,0.042711,0.0315,133.980,C,0.039347,1.028057,4.768374e-07
100,32903.50,33000.0,121.1,0.000822,0.018428,0.024842,0.0180,129.580,C,0.021635,0.997076,3.582755e-02
111,32903.49,34000.0,130.0,0.000822,0.007749,0.018371,0.0080,127.485,C,0.013060,0.967750,3.195586e-01
29,32928.40,35000.0,155.3,0.000822,0.002462,0.006331,0.0035,125.765,C,0.004397,0.940811,5.370901e-01
180,32898.63,36000.0,62.8,0.000822,0.000887,0.004693,0.0015,128.710,C,0.002790,0.913851,7.583158e-01


In [25]:
option.append_loc_vol_to_df()

/Users/pierrebelamri/Documents/GitHub/deribit_scraping/option.py:140: RuntimeWarning:

divide by zero encountered in double_scalars

/Users/pierrebelamri/Documents/GitHub/deribit_scraping/option.py:140: RuntimeWarning:

invalid value encountered in sqrt



In [26]:
option.append_BS_price()

In [27]:
option.df.dropna(inplace=True)
#option.df = option.df.drop(option.df[option.df.BS_PRICE >= 1.0].index)

In [28]:
import plotly.graph_objects as go

x = option.df['K']
y = option.df['_T']
z = option.df['BS_PRICE']
z2 = option.df['mid']

"""
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z , mode='markers', name='BS price')])
"""
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z , mode='markers', name='Black&Scholes model Price'),
                      go.Scatter3d(x=x, y=y, z=z2, mode='markers', name='Real Market Price')])


fig.update_scenes(xaxis_title_text='Strike', yaxis_title_text='Exp', zaxis_title_text='Price') 
fig.show()

In [29]:
rms = mean_squared_error(option.df['BS_PRICE'], option.df['mid'], squared=False)
print('Root Mean Squared Error = {}%'.format(round(rms*100,3)))

Root Mean Squared Error = 1.091%


In [30]:
option.init_merton(m = 0.4, lam =1, v = 0.3,  )

In [31]:
option.append_Merton_price()

In [32]:
option.df.dropna(inplace=True)
#option.df = option.df.drop(option.df[option.df.MERTON_PRICE >= .2].index)

In [33]:
option.optimize_merton()

Calibrated Jump Mean =  1.0352256226434133
Calibrated Jump Std =  1.0068365141530089e-05
Calibrated intensity =  0.0


In [36]:
import plotly.graph_objects as go

option.append_Merton_price()

x = option.df['K']
y = option.df['_T']
z = option.df['BS_PRICE']
z2 = option.df['mid']
z3 = option.df['MERTON_PRICE']


#fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z , mode='markers', name='BS price')])

fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z , mode='markers', name='Black&Scholes model Price'),
                      go.Scatter3d(x=x, y=y, z=z2, mode='markers', name='Real Market Price'),
                      go.Scatter3d(x=x, y=y, z=z3, mode='markers', name='Merton Model Price')])


fig.update_scenes(xaxis_title_text='Strike', yaxis_title_text='Exp', zaxis_title_text='Price') 
fig.show()

rms_merton = mean_squared_error(option.df['mid'], option.df['MERTON_PRICE'], squared=False)
rms_bs = mean_squared_error(option.df['mid'], option.df['BS_PRICE'], squared=False)

print('Root Mean Squared Error | MERTON = {}%'.format(rms_merton*100))
print('Root Mean Squared Error | B&S    = {}%'.format(rms_bs*100))

Root Mean Squared Error | MERTON = 9073.198438652968%
Root Mean Squared Error | B&S    = 1.090642756694224%
